In [63]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import label as lab
from skimage.measure import regionprops
from skimage.morphology import binary_erosion
from skimage.measure import regionprops_table
from skimage.measure import moments_hu
from skimage.color import rgb2gray
from skimage import img_as_ubyte
import numpy as np
import cv2
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
import joblib
import json
from hsv import HsvExtractor
import numpy as np
import cv2
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
import joblib
import json
from hsv import HsvExtractor
import os

def extract_features(image):
    features = []
    
    # Ubah gambar menjadi grayscale
    gray_image = rgb2gray(image)
    gray_image = img_as_ubyte(gray_image)
    
    # Metric: jumlah area piksel yang bernilai 1
    labeled_image = lab(gray_image > 0)
    # print(type(labeled_image))
    props = regionprops(labeled_image)
    # print(type(props))
    area = props[0].area if props else 0
    # print(type(area))
    features.append(area)
    
    # Eccentricity: eksentrisitas region
    if props:
        eccentricity = props[0].eccentricity
    else:
        eccentricity = 0
    features.append(eccentricity)
    
    # GLCM (Gray Level Co-occurrence Matrix)
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    features.append(contrast)
    features.append(energy)
    
    return features

In [64]:
data_dir = 'OUPUT_DATASET/data latih'
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]

# Load images and extract features
X = []
y = []
label_mapping = {}  # Dictionary untuk memetakan angka ke label
feature_extractor = HsvExtractor()
for idx, image_file in enumerate(image_files):
    # print(idx)
    img = cv2.imread(image_file)
    image = extract_features(img)
    X.append(image)
    # Dapatkan label dari nama berkas (misal: 'Ayam Tidak Segar_1.jpg' menjadi 'Ayam Tidak Segar')
    label = image_file.split('/')[-1].split('_')[0]
    if label not in label_mapping:
        label_mapping[label] = idx  # Setiap label memiliki angka unik
    y.append(label_mapping[label])

# Export dictionary label_mapping
with open('label_mapping.json', 'w') as json_file:
    json.dump(label_mapping, json_file)

X_train = X
y_train = y

In [65]:
data_dir = 'OUPUT_DATASET/data testing'
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]

# Load images and extract features
X = []
y = []
# label_mapping = {}  # Dictionary untuk memetakan angka ke label
# feature_extractor = HsvExtractor()
for idx, image_file in enumerate(image_files):
    # print(idx)
    # crop_object(image_file)
    img = cv2.imread(image_file)
    image = extract_features(img)
    X.append(image)
    # Dapatkan label dari nama berkas (misal: 'Ayam Tidak Segar_1.jpg' menjadi 'Ayam Tidak Segar')
    label = image_file.split('/')[-1].split('_')[0]
    if label not in label_mapping:
        label_mapping[label] = idx  # Setiap label memiliki angka unik
    y.append(label_mapping[label])

# # Export dictionary label_mapping
# with open('label_mapping.json', 'w') as json_file:
#     json.dump(label_mapping, json_file)

X_test = X
y_test = y

In [ ]:
# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [66]:
X_train_features = np.array([image for image in X_train])
X_test_features = np.array([image for image in X_test])

In [67]:
# KNN classification
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_features, y_train)

# Save KNN model to file
pickle.dump(knn, open('KNN.pkl', 'wb'))

In [68]:
# Evaluate KNN model
knn_preds = knn.predict(X_test)
knn_rmse = mean_squared_error(y_test, knn_preds, squared=False)
print('KNN RMSE:', knn_rmse)

# tampikan akurasinya
print('Akurasi KNN:', knn.score(X_test, y_test))

# Save KNN model to file
with open('KNN.pkl', 'wb') as knn_model_file:
    pickle.dump(knn, knn_model_file)

KNN RMSE: 29.725410005582766
Akurasi KNN: 0.0


In [75]:
import os
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import regionprops, label
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pickle
import json

# Fungsi untuk mengekstrak fitur dari gambar
def extract_features(image):
    features = []

    # Mengubah gambar menjadi grayscale
    gray_image = rgb2gray(image)
    gray_image = (gray_image * 255).astype(np.uint8)

    # Metric: Jumlah area piksel yang bernilai 1
    labeled_image = label(gray_image > 0)
    props = regionprops(labeled_image)
    area = props[0].area if props else 0
    features.append(area)

    # Eccentricity: Eksentrisitas region
    if props:
        eccentricity = props[0].eccentricity
    else:
        eccentricity = 0
    features.append(eccentricity)

    # GLCM (Gray Level Co-occurrence Matrix)
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    features.append(contrast)
    features.append(energy)

    return features

# Membaca dataset dari folder Latih dan folder Uji
def load_dataset(folder_path):
    images = []
    labels = []
    label_mapping = {}  # Dictionary untuk memetakan angka ke label

    for idx, filename in enumerate(os.listdir(folder_path)):
        if filename.endswith('.jpg'):
            # Mengambil label dari nama berkas
            label = filename.split(' ')[0]
            labels.append(label)

            # Membaca dan mengekstrak fitur gambar
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            features = extract_features(image)
            images.append(features)

            # Menambahkan label ke dalam dictionary jika belum ada
            if label not in label_mapping:
                label_mapping[label] = idx  # Setiap label memiliki angka unik

    return images, labels, label_mapping

# Memuat dataset latih dan uji
latih_images, latih_labels, label_mapping_latih = load_dataset('OUPUT_DATASET/data latih')
uji_images, uji_labels, label_mapping_uji = load_dataset('OUPUT_DATASET/data testing')

# Membagi dataset latih menjadi data latih dan data validasi
X_train, X_val, y_train, y_val = train_test_split(latih_images, latih_labels, test_size=0.2, random_state=42)

# Menginisialisasi dan melatih model KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Melakukan prediksi pada data validasi
y_pred = knn.predict(X_val)

# Menghitung akurasi model
accuracy = accuracy_score(y_val, y_pred)
print(f"Akurasi KNN: {accuracy * 100:.2f}%")

# Save KNN model to file
with open('KNN.pkl', 'wb') as knn_model_file:
    pickle.dump(knn, knn_model_file)

# Export label_mapping ke dalam JSON
with open('label_mapping.json', 'w') as json_file:
    json.dump(label_mapping_uji, json_file)

# Melakukan prediksi pada data uji
y_uji_pred = knn.predict(uji_images)



Akurasi KNN: 55.00%


In [1]:
import os
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import regionprops, label
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pickle
import json
import pandas as pd

def normalize(data, min_value, max_value):
    """
    Normalizes a float to the range [0, 1].

    Args:
        data: The float to be normalized.
        min_value: The minimum value in the input data.
        max_value: The maximum value in the input data.

    Returns:
        A normalized float.
    """

    # Check for invalid input

    if not isinstance(data, float):
        raise ValueError("data must be a float")

    if min_value >= max_value:
        raise ValueError("min_value must be less than max_value")

    # Calculate the range of the input data

    range_value = max_value - min_value

    # Normalize the input data

    normalized_data = (data - min_value) / range_value

    return normalized_data

# Fungsi untuk mengekstrak fitur dari gambar
def extract_features(image):
    features = []

    # Mengubah gambar menjadi grayscale
    gray_image = rgb2gray(image)
    gray_image = (gray_image * 255).astype(np.uint8)

    # Metric: Jumlah area piksel yang bernilai 1
    labeled_image = label(gray_image > 0)
    props = regionprops(labeled_image)
    area = props[0].area if props else 0

    # Eccentricity: Eksentrisitas region
    if props:
        eccentricity = props[0].eccentricity
    else:
        eccentricity = 0

    # # Gabungkan Metric dan Eccentricity menjadi satu fitur
    # combined_feature = [
    #     normalize(area, 86070.0, 3288384.0), 
    #     normalize(eccentricity, 0.052746550212344645, 0.9687462317467702)]
    # features.extend(combined_feature)

    # GLCM (Gray Level Co-occurrence Matrix)
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]

    # Ekstraksi fitur entropy dan homogenitas
    entropy = -np.sum(glcm * np.log(glcm + np.finfo(float).eps))
    homogeneity = np.sum(glcm / (1.0 + np.abs(0 - 1)))

     # Normalisasi fitur
    normalized_area = normalize(area, 86070.0, 3288384.0)
    normalized_eccentricity = normalize(eccentricity, 0.052746550212344645, 0.9687462317467702)
    normalized_contrast = normalize(contrast, 3.09712535728869, 379.15949047118636)
    normalized_energy = normalize(energy, 0.0303806915008107, 0.6785592519223528)
    normalized_entropy = normalize(entropy, 0, 7)  # Rentang nilai entropy biasanya [0, 7]
    normalized_homogeneity = normalize(homogeneity, 0, 1)  # Rentang nilai homogenitas biasanya [0, 1]

    # Gabungkan semua fitur menjadi satu
    combined_feature = [
        normalized_area, normalized_eccentricity,
        normalized_contrast, normalized_energy,
        normalized_entropy, normalized_homogeneity
    ]

    features.extend(combined_feature)
    

    # features = [i / 255 for i in features]
    # print(features)

    return features

# Membaca dataset dari folder Latih dan folder Uji
def load_dataset(folder_path):
    images = []
    labels = []
    label_mapping = {}  # Dictionary untuk memetakan angka ke label

    for idx, filename in enumerate(os.listdir(folder_path)):
        if filename.endswith('.jpg'):
            # Mengambil label dari nama berkas
            label = filename.split(' ')[0]
            labels.append(label)

            # Membaca dan mengekstrak fitur gambar
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            features = extract_features(image)
            images.append(features)

            # Menambahkan label ke dalam dictionary jika belum ada
            if label not in label_mapping:
                label_mapping[label] = idx  # Setiap label memiliki angka unik

    return images, labels, label_mapping

# Memuat dataset latih dan uji
latih_images, latih_labels, label_mapping_latih = load_dataset('OUPUT_DATASET/Latih')
uji_images, uji_labels, label_mapping_uji = load_dataset('OUPUT_DATASET/Uji')

# Fungsi untuk melakukan klasifikasi menggunakan KNN dengan fitur yang diberikan
def classify_and_evaluate(features, labels, knn_neighbors, name_model):
    # Membagi dataset latih menjadi data latih dan data validasi
    X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Menginisialisasi dan melatih model KNN
    knn = KNeighborsClassifier(n_neighbors=knn_neighbors)
    knn.fit(X_train, y_train)

    # Melakukan prediksi pada data validasi
    y_pred = knn.predict(X_val)

    # Menghitung akurasi model
    accuracy = accuracy_score(y_val, y_pred)
    # Save KNN model to file
    with open(f'{name_model}.pkl', 'wb') as knn_model_file:
        pickle.dump(knn, knn_model_file)

    # Export label_mapping ke dalam JSON
    with open(f'label_mapping_{name_model}.json', 'w') as json_file:
        json.dump(label_mapping_uji, json_file)

    return accuracy

def normalize(features):
    # Mencari nilai minimum dan maksimum untuk setiap fitur
    min_values = []
    max_values = []
    for feature in features:
        min_values.append(min(feature))
        max_values.append(max(feature))

    # Melakukan normalisasi
    for i, feature in enumerate(features):
        features[i] = (feature - min_values[i]) / (max_values[i] - min_values[i])

    return features

# Ekstraksi fitur untuk masing-masing jenis
features_sets = {
    'Eccentricity Metric': [feat[:2] for feat in latih_images],  # Menggunakan gabungan Eccentricity dan Metric
    'GLCM': [feat[2:] for feat in latih_images]
}

def flatten_list(list_of_lists):
    flat_list = []
    for sublist in list_of_lists:
        flat_list.extend(sublist)

    return flat_list



In [2]:
features_sets2 = {
    'Eccentricity Metric': features_sets['Eccentricity Metric'],  # Menggunakan gabungan Eccentricity dan Metric
    'GLCM Contrast': flatten_list(features_sets['GLCM Contrast']),
    'GLCM Energy': flatten_list(features_sets['GLCM Energy'])
}

KeyError: 'GLCM Contrast'

In [14]:
# GLCM Contrast
col = 'GLCM Contrast'
print(min(features_sets2[col]))
print(max(features_sets2[col]))

3.09712535728869
379.15949047118636


In [15]:
# GLCM Contrast
col = 'GLCM Energy'
print(min(features_sets2[col]))
print(max(features_sets2[col]))

0.0303806915008107
0.6785592519223528


In [19]:
features_sets2[col][1]

[191878.0, 0.8280297669312676]

In [23]:
# GLCM Contrast
col = 'Eccentricity Metric'
feat1 = [feat[0] for feat in features_sets2[col]]
feat2 = [feat[1] for feat in features_sets2[col]]
print(f"feat1\n=> min : {min(feat1)}\n=> max : {max(feat1)}")
print(f"feat2\n=> min : {min(feat2)}\n=> max : {max(feat2)}")

feat1
=> min : 86070.0
=> max : 3288384.0
feat2
=> min : 0.052746550212344645
=> max : 0.9687462317467702


In [3]:
# Klasifikasi dan evaluasi untuk setiap jenis fitur
for feature_name, feature_set in features_sets.items():
    accuracy = classify_and_evaluate(feature_set, latih_labels, 3, feature_name)
    print(f"Akurasi KNN dengan fitur {feature_name}: {accuracy * 100:.2f}%")

Akurasi KNN dengan fitur Eccentricity Metric: 60.00%
Akurasi KNN dengan fitur GLCM: 46.00%


In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Fungsi untuk mengekstrak fitur dari gambar
def extract_features(image):
    # Mengubah gambar menjadi grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize gambar menjadi ukuran 32x32
    resized_image = cv2.resize(gray_image, (32, 32))

    # Mengembalikan fitur gambar
    return resized_image

# Membaca dataset dari folder Latih dan folder Uji
def load_dataset(folder_path):
    images = []
    labels = []

    for idx, filename in enumerate(os.listdir(folder_path)):
        if filename.endswith('.jpg'):
            # Mengambil label dari nama berkas
            label = filename.split(' ')[0]
            labels.append(label)

            # Membaca dan mengekstrak fitur gambar
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            features = extract_features(image)
            images.append(features)

    return images, labels

# Memuat dataset latih dan uji
latih_images, latih_labels = load_dataset('OUPUT_DATASET/Latih')
uji_images, uji_labels = load_dataset('OUPUT_DATASET/Uji')

# Mengubah label string menjadi integer
le = LabelEncoder()
le.fit(latih_labels)
latih_labels = le.transform(latih_labels)
uji_labels = le.transform(uji_labels)

# Mengubah label menjadi one-hot encoding
latih_labels = to_categorical(latih_labels)
uji_labels = to_categorical(uji_labels)

# Membagi dataset latih menjadi data latih dan data validasi
X_train, X_val, y_train, y_val = train_test_split(latih_images, latih_labels, test_size=0.2, random_state=42)

# Membangun model CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(latih_labels[0]), activation='softmax'))

# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Melatih model
model.fit(X_train, y_train, epochs=10)

# Mengevaluasi model pada data validasi
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Loss validasi: {val_loss:.4f}, Akurasi validasi: {val_accuracy:.2f}%')

# Menyimpan model
model.save('model_cnn.h5')

ValueError: Data cardinality is ambiguous:
  x sizes: 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32
  y sizes: 200
Make sure all arrays contain the same number of samples.

In [76]:
# Export label_mapping ke dalam JSON
with open('label_mapping.json', 'w') as json_file:
    json.dump(label_mapping_latih, json_file)

In [77]:
def reverse_dict(d):
    return {v: k for k, v in d.items()}

In [79]:
label_mapping_latih2 = reverse_dict(label_mapping_uji)
# Menampilkan hasil prediksi pada data uji
for i in range(len(uji_images)):
    label = uji_labels[i]
    label_index = label_mapping_uji[label]
    label_name = label_mapping_latih2[label_index]
    print(f"Gambar: {label}, Prediksi: {y_uji_pred[i]}, Label Asli: {label_name}")


Gambar: Cemara, Prediksi: Ginko, Label Asli: Cemara
Gambar: Cemara, Prediksi: Sambiloto, Label Asli: Cemara
Gambar: Cemara, Prediksi: Cemara, Label Asli: Cemara
Gambar: Cemara, Prediksi: Cemara, Label Asli: Cemara
Gambar: Cemara, Prediksi: Maja, Label Asli: Cemara
Gambar: Emas, Prediksi: Ginko, Label Asli: Emas
Gambar: Emas, Prediksi: Emas, Label Asli: Emas
Gambar: Emas, Prediksi: Emas, Label Asli: Emas
Gambar: Emas, Prediksi: Emas, Label Asli: Emas
Gambar: Emas, Prediksi: Emas, Label Asli: Emas
Gambar: Ginko, Prediksi: Ginko, Label Asli: Ginko
Gambar: Ginko, Prediksi: Emas, Label Asli: Ginko
Gambar: Ginko, Prediksi: Emas, Label Asli: Ginko
Gambar: Ginko, Prediksi: Ginko, Label Asli: Ginko
Gambar: Ginko, Prediksi: Emas, Label Asli: Ginko
Gambar: Lemon, Prediksi: Emas, Label Asli: Lemon
Gambar: Lemon, Prediksi: Pegagan, Label Asli: Lemon
Gambar: Lemon, Prediksi: Lemon, Label Asli: Lemon
Gambar: Lemon, Prediksi: Maja, Label Asli: Lemon
Gambar: Lemon, Prediksi: Maja, Label Asli: Lemon
Gam

In [56]:
image_ext = 'dataset\Latih\Cemara Kipas 1.jpg'


def crop_object(image_ext, output):
    import cv2
    import numpy as np
    #img = cv2.imread("test.png")
    img = cv2.imread(image_ext)
    blurred = cv2.blur(img, (5,5))
    canny = cv2.Canny(blurred, 50, 200)

    ## find the non-zero min-max coords of canny
    pts = np.argwhere(canny>0)
    y1,x1 = pts.min(axis=0)
    y2,x2 = pts.max(axis=0)

    ## crop the region
    cropped = img[y1:y2, x1:x2]
    cv2.imwrite(output, cropped)

    # tagged = cv2.rectangle(img.copy(), (x1,y1), (x2,y2), (0,255,0), 3, cv2.LINE_AA)
    # cv2.imwrite('hasil_crop.jpg', tagged)
    # cv2.imshow("tagged", tagged)
    # cv2.waitKey()


In [ ]:
import cv2
import numpy as np
from PIL import Image

# Mengimpor model Ultra-Analytix YOLO
from ultralytics import YOLO

# Membaca citra
image = cv2.imread(image_ext)

# Membuat objek Ultra-Analytix YOLO
model = YOLO('yolov8n.pt')

# Mendeteksi objek pada citra
results = model.detect(source=Image.open(image_ext), save=True)

# Mendapatkan hasil deteksi
for result in results:
    class_id, confidence, box = result
    class_name = model.class_names[class_id]

    # Menggambar kotak dan label pada objek yang terdeteksi
    x, y, w, h = box
    color = (0, 255, 0)  # Warna hijau
    thickness = 2
    cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)
    cv2.putText(image, f'{class_name}: {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

# Menyimpan citra dengan objek yang terdeteksi
cv2.imwrite('hasil_deteksi.jpg', image)

# Menampilkan hasil deteksi
cv2.imshow('Hasil Deteksi', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [62]:
data_dir = 'dataset/Uji'
output_dataset = 'OUPUT_DATASET/'
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]

# Load images and extract features
X = []
y = []
# label_mapping = {}  # Dictionary untuk memetakan angka ke label
# feature_extractor = HsvExtractor()
for idx, image_file in enumerate(image_files):
    # print(idx)
    label = image_file.split('/')[-1].split(' ')[0]
    crop_object(image_file, f"{output_dataset}{label} {idx}.jpg")

In [1]:
import os
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import regionprops, label
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pickle
import json

def crop_object(img):
    blurred = cv2.blur(img, (5,5))
    canny = cv2.Canny(blurred, 50, 200)

    ## find the non-zero min-max coords of canny
    pts = np.argwhere(canny>0)
    y1,x1 = pts.min(axis=0)
    y2,x2 = pts.max(axis=0)

    ## crop the region
    cropped = img[y1:y2, x1:x2]
    return cropped

# Fungsi untuk mengekstrak fitur dari gambar
def extract_features(image):
    features = []

    # Mengubah gambar menjadi grayscale
    gray_image = rgb2gray(image)
    gray_image = (gray_image * 255).astype(np.uint8)

    # Metric: Jumlah area piksel yang bernilai 1
    labeled_image = label(gray_image > 0)
    props = regionprops(labeled_image)
    area = props[0].area if props else 0
    features.append(area)

    # Eccentricity: Eksentrisitas region
    if props:
        eccentricity = props[0].eccentricity
    else:
        eccentricity = 0
    features.append(eccentricity)

    # GLCM (Gray Level Co-occurrence Matrix)
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    features.append(contrast)
    features.append(energy)

    return features

# Membaca dataset dari folder Latih dan folder Uji
def load_dataset(folder_path):
    images = []
    labels = []
    label_mapping = {}  # Dictionary untuk memetakan angka ke label

    for idx, filename in enumerate(os.listdir(folder_path)):
        if filename.endswith('.jpg'):
            # Mengambil label dari nama berkas
            label = filename.split(' ')[0]
            labels.append(label)

            # Membaca dan mengekstrak fitur gambar
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            features = extract_features(crop_object(image))
            images.append(features)

            # Menambahkan label ke dalam dictionary jika belum ada
            if label not in label_mapping:
                label_mapping[label] = idx  # Setiap label memiliki angka unik

    return images, labels, label_mapping

# Memuat dataset latih dan uji
latih_images, latih_labels, label_mapping_latih = load_dataset('dataset/Latih')
uji_images, uji_labels, label_mapping_uji = load_dataset('dataset/Uji')

# Membagi dataset latih menjadi data latih dan data validasi
X_train, X_val, y_train, y_val = train_test_split(latih_images, latih_labels, test_size=0.2, random_state=42)

# Menginisialisasi dan melatih model KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Melakukan prediksi pada data validasi
y_pred = knn.predict(X_val)

# Menghitung akurasi model
accuracy = accuracy_score(y_val, y_pred)
print(f"Akurasi KNN: {accuracy * 100:.2f}%")

# Save KNN model to file
with open('KNN.pkl', 'wb') as knn_model_file:
    pickle.dump(knn, knn_model_file)

# Export label_mapping ke dalam JSON
with open('label_mapping.json', 'w') as json_file:
    json.dump(label_mapping_uji, json_file)

# Melakukan prediksi pada data uji
y_uji_pred = knn.predict(uji_images)

# Export label_mapping ke dalam JSON
with open('label_mapping.json', 'w') as json_file:
    json.dump(label_mapping_latih, json_file)

def reverse_dict(d):
    return {v: k for k, v in d.items()}

label_mapping_latih2 = reverse_dict(label_mapping_uji)
# Menampilkan hasil prediksi pada data uji
for i in range(len(uji_images)):
    label = uji_labels[i]
    label_index = label_mapping_uji[label]
    label_name = label_mapping_latih2[label_index]
    print(f"Gambar: {label}, Prediksi: {y_uji_pred[i]}, Label Asli: {label_name}")


Akurasi KNN: 55.00%
Gambar: Cemara, Prediksi: Ginko, Label Asli: Cemara
Gambar: Cemara, Prediksi: Sambiloto, Label Asli: Cemara
Gambar: Cemara, Prediksi: Cemara, Label Asli: Cemara
Gambar: Cemara, Prediksi: Cemara, Label Asli: Cemara
Gambar: Cemara, Prediksi: Maja, Label Asli: Cemara
Gambar: Emas, Prediksi: Ginko, Label Asli: Emas
Gambar: Emas, Prediksi: Emas, Label Asli: Emas
Gambar: Emas, Prediksi: Emas, Label Asli: Emas
Gambar: Emas, Prediksi: Emas, Label Asli: Emas
Gambar: Emas, Prediksi: Emas, Label Asli: Emas
Gambar: Ginko, Prediksi: Ginko, Label Asli: Ginko
Gambar: Ginko, Prediksi: Emas, Label Asli: Ginko
Gambar: Ginko, Prediksi: Emas, Label Asli: Ginko
Gambar: Ginko, Prediksi: Ginko, Label Asli: Ginko
Gambar: Ginko, Prediksi: Emas, Label Asli: Ginko
Gambar: Lemon, Prediksi: Emas, Label Asli: Lemon
Gambar: Lemon, Prediksi: Pegagan, Label Asli: Lemon
Gambar: Lemon, Prediksi: Lemon, Label Asli: Lemon
Gambar: Lemon, Prediksi: Maja, Label Asli: Lemon
Gambar: Lemon, Prediksi: Maja, L